In [2]:
from dataset import EmbeddingDataset
from attention import MultiScaleTemporalAttention

dataset = EmbeddingDataset("embeddings/embeddings_50.pkl")
len(dataset)

136

In [5]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=4)
len(dataloader)

34

In [8]:
attn = MultiScaleTemporalAttention(768, 2)
attn(next(iter(dataloader))).shape

torch.Size([4, 50, 768])